In [46]:
import gymnasium as gym
from adaptive_time.features import Fourier_Features
import numpy as np

In [92]:
env = gym.make('CartPole-v1')
tau = 0.02
env.stepTime(tau)
env.reset()
def generate_trajectory(env):
    observation, _ = env.reset()
    trajectory = []
    terminated = False
    for _ in range(1000):
        action = 1
        observation_, reward, terminated, truncated, info = env.step(action)
        trajectory.append([observation, action, reward, observation_])
        observation = observation_

        if terminated or truncated:
            return trajectory

    return trajectory

trajectory = generate_trajectory(env)

In [93]:
phi = Fourier_Features()
phi.init_fourier_features(4,1)
phi.init_state_normalizers(np.array([4.8,2.0,0.418,1]), np.array([-4.8,-2.0,-0.418,-1]))


def foo2(xs,tol,level):
    c=int(np.floor(len(xs)/2))
    #print(xs, c)
    f = lambda xs: len(xs)*(xs[0]+xs[-1])/2 if len(xs) else 0
    if abs(f(xs) - (r:=f(xs[:c]) + f(xs[c:]))) < tol: return 1, r, [level+c]
    else: 
        x, a, cs_a = foo2(xs[:c],tol/2, level)
        y, b, cs_b = foo2(xs[c:],tol/2, level+c)
        return x+y+1, a+b, cs_a + cs_b + [level+c]

In [94]:
rewards = []
for lists in trajectory:
    rewards.append(lists[2])

a, b, c = foo2(rewards, 0.1, 0)

In [95]:
print(d)

16


In [98]:


def ols_monte_carlo(env, phi, weights, targets, features, x0, gamma = 0.999):
    trajectory = generate_trajectory(env)
    d = len(weights)
    N = len(trajectory)
    G = 0
    returns = []
    points = np.linspace(0,N-1,d)
    for t in range(N-1,-1,-1):
        state, _, reward, _ = trajectory[t]
        x = phi.get_fourier_feature(state)
        G = gamma * G + reward
        if np.linalg.norm(x-x0) < 0.001:
            returns.append(G)
    
    G = 0
    for t in points:
        t = int(t)
        state, _, reward, _ = trajectory[t]
        G = gamma*G + reward
        x = phi.get_fourier_feature(state)
        features += np.outer(x,x)
        targets += G * x
    weights = np.linalg.solve(features, targets)
    return weights, targets, features, np.mean(returns)


def gradient_monte_carlo(env, phi, weights, x0, gamma = 1, alpha = 0.001):
    trajectory = generate_trajectory(env)
    N = len(trajectory)
    G = 0
    returns = []
    for t in range(N-1,-1,-1):
        state, _, reward, _ = trajectory[t]
        G = gamma * G + reward
        x = phi.get_fourier_feature(state)
        if np.linalg.norm(x-x0) < 0.00001:
            returns.append(G)
        weights = weights + alpha * (G - np.inner(x, weights)) * x 
    return weights, np.mean(returns)




num_episodes = 10000
Returns = []
observation, _ = env.reset()
d = len(phi.get_fourier_feature(observation))
features = np.identity(d)
targets = np.zeros(d)
weights = np.zeros(d)
x = phi.get_fourier_feature([0,0,0,0])
for episode in range(num_episodes):
    weights, targets, features, returns = ols_monte_carlo(env, phi, weights, targets, features, x)
    if returns:
        Returns.append(returns)
    if Returns:
        print('episode:',episode, ' empirical returns:' , np.mean(Returns)/np.inner(x,weights), ' predicted returns:' , np.inner(x,weights))

episode: 0  empirical returns: 8.407856614765258  predicted returns: 1.5369297882164308
episode: 1  empirical returns: 8.404111544528844  predicted returns: 1.5376146803641384
episode: 2  empirical returns: 8.374353830590733  predicted returns: 1.5430784927048795
episode: 3  empirical returns: 8.352636832000458  predicted returns: 1.5470905231719974
episode: 4  empirical returns: 8.336933459478017  predicted returns: 1.5500046089002084
episode: 5  empirical returns: 8.325101752364725  predicted returns: 1.552207488949278
episode: 6  empirical returns: 8.3158443753197  predicted returns: 1.553935439753644
episode: 7  empirical returns: 8.308383133913935  predicted returns: 1.5553309323853748
episode: 8  empirical returns: 8.302231356547432  predicted returns: 1.5564833996217553
episode: 9  empirical returns: 8.297070155522285  predicted returns: 1.55745161172159
episode: 10  empirical returns: 8.29268142566226  predicted returns: 1.5582758607241807
episode: 11  empirical returns: 8.2889

KeyboardInterrupt: 

In [ ]:
def ols_monte_carlo_control(env, phi, weights, targets, features, x0, gamma = 199/200):
    trajectory = generate_trajectory(env)
    N = len(trajectory)
    G = 0
    returns = []
    for t in range(N-1,-1,-1):
        state, action, reward, _ = trajectory[t]
        G = gamma*G + reward
        x = phi.get_fourier_feature(state)
        features[action] += np.outer(x,x)
        targets[action] += G * x
    for action in range(2)
    weights = np.linalg.solve(features, targets)
    return weights, targets, features, G

LinAlgError: Singular matrix